# Moscot map to growth/apoptosis rates

This is a short script that loads computed moscot maps to inspect the growth rates and the apoptosis rate

In [ ]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy


from moscot.problems.time._lineage import TemporalProblem

In [2]:
Path="/home/mgander/moscot-framework_reproducibility/data/mouse_temporal"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

In [3]:
# This is my environment where moscot runs on CPU
pip freeze

absl-py @ file:///home/conda/feedstock_root/build_artifacts/absl-py_1673535674859/work
aiohttp==3.8.4
aiosignal==1.3.1
anndata==0.9.1
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-timeout==4.0.2
attrs==23.1.0
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
certifi==2022.12.7
charset-normalizer==3.1.0
chex @ file:///home/conda/feedstock_root/build_artifacts/chex_1679925560941/work
cloudpickle==2.2.1
cmake==3.26.3
comm @ file:///croot/comm_1671231121260/work
contextlib2==21.6.0
contourpy==1.0.7
cycler @ file:///home/conda/feedstock_root/build_artifacts/cycler_1635519461629/work
debugpy @ file:///home/builder/ci_310/debugpy_1640789504635/work
decorator @ file:///opt/conda/conda-bld/decorator_1643638310831/work
dm-tree @ file:///home/conda/feedstock_root/build_artifacts/dm-tree_1649681253781/work
dnspython==2.3.0
docrep==0.3.2
entrypoints @ file:///tmp/build/80754af9/entrypoints_1649908313000/work
et-xmlfile==1.1.0
executing @ file:///opt/cond

# Moscot maps to growth rates

In [3]:
cells=[32, 80, 100, 120, 400, 660, 1720, 4500, 8200, 15000, 30000, 60000, 73000, 90000, 90000, 200000, 1100000, 2600000, 6000000, 13000000]
Cell_number_dict={}
for i in range(20):
    Cell_number_dict[ts[i]]=cells[i]

In [4]:
# Doesn't matter which adata you use to create a tp instance, the result is always the same. I just need one
# to instantiate a tp-problem
adata=sc.read(f"{Path}/anndatas/Comb_anndatas/adata_E3.5_E4.5.h5ad")
tp=TemporalProblem(adata)

Only considering the two last: ['.5', '.h5ad'].
Only considering the two last: ['.5', '.h5ad'].


In [5]:
i=13
ts0=ts[i]
ts1=ts[i+1]
print(f'{ts0} --> {ts1}')
print('------------------------')

E12.5 --> E13.5
------------------------


In [6]:
# I ignored constant folding errors, results looked good

tp=tp.load(f'{Path}/moscot_maps_cell_type_transition_analysis/{ts0}_{ts1}_TemporalProblem.pkl')
adata=tp.adata
time_tuple=list(tp.solutions.keys())[0]
gr=tp[time_tuple].solution.a
gr=gr/gr.mean()*Cell_number_dict[ts1]/Cell_number_dict[ts0]
cell_dying=np.sum((1-gr[gr<1]))
apoptosis_rate=float(cell_dying/len(gr))
apoptosis_rate

2023-05-01 10:33:57.264447: E external/xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  reduce-window.6 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2023-05-01 10:33:57.416107: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.151915895s
Constant folding an instruction is taking > 1s:

  reduce-window.6 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a 

0.021524176001548767

In [ ]:
apoptosis_rate